# Experimental ML with HoloViz + PyTorch in Jupyterlab
A guide through multi-class road detection on satellite images with interactive visualization and explorative model building

- type: PyData LA 2019 Proposal
- date: 2019-09-21
- author: Hayley Song (haejinso@usc.edu)
- Category: step-by-step tutorial
- Prereq: 
    - Basic understanding of visaulization with python (eg. previously have used matplotlib.pyplot library)
    - Basic understanding of neural network training process   
    I'll give a brief overview of the workflow, assuming audiences' previous experience with the following concepts
        - mini-batch training
        - forward-pass, backword-pass 
        - gradient, gradient descent algorithm
        - classification, semantic segmentation
        - image as numpy ndarray
- Material distribution
    - All materials needed to follow the tutorial will be shared in a self-containing GitHub repo, as well as a Binder environment
    - I will create a docker image with the required libraries installed
    - Links to extra resources will be provided as appropriate

## Overview + Goal

This tutorial introduces how to make your data exploration and model building process more interactive and exploratory by using the combination of JupyterLab, HoloViz, and PyTorch.  [HoloViz](https://holoviz.org/) is a set of Python libraries that offers simple yet powerful visualization and GUI building tools which, together with other data analysis libraries (eg. `pandas`, `geopandas`, `numpy`) and machine learning framework (eg. `PyTorch`, `Tensorflow`) can make your modeling procedure more interactive and exploratory.   I will start by introducing the four core HoloViz libraries (Holoviews, GeoViews, Panel and Param) and demonstrate basic examples on how we can essentially replace any "Matplotlib.pyplot" calls with equivalents in `HoloViz`.  You will see how this opens up the possibilities to directly interact with your visualization by eg. hovering over the graph to inspect values, querying RGB values of an image, or Lat/Lon values on your map.

Following the introduction of the HoloViz libraries, I will demonstrate how to embellish each step of your machine learning workflow with HoloViz. First, you will learn to easily turn your PyTorch codes into a simple GUI that encaptulates the state of your model (or alternatively, the state of your training session). This GUI explicitly exposes your model parameters and training hyperparameters (eg. learning rate, optimizer settings, batch size) as directly tunable parameters.  Compared to conventional ways of specifying the hyperparameter settings with the help of 'argparse' library or config files, this GUI approach focuses on the experimental nature of modeling and integrates seamlessly with Jupyter notebooks.  After training a neural network model using our own GUI in the notebook, I will demonstrate how to understand the model by visualizing the intermediate layers with HoloViz and test the model with test images directly sampled from HoloViz visualization. 

To illustrate these steps, I will focus on the problem of classfying different types of roads on satellite images, defined as a multi-class semantic segmentation problem.  Starting from the data exploration to the trained model understanding, you will learn different ways to explore their data and models by easily building simple GUIs in a Jupyter notebook. 

In summary, by the end of the talk the audience will have learned:
- how to make their data exploration more intuitive and experimental by using HoloViz libraries
- how to turn their codes into a simple GUI that allows the above
- how to that exposes 
the benefits of using `Holoviews`/`Geoviews` for interactively exploring datasets and monitoring their training process
- how to use `param` and `panel` to define a neural network whose parameters are configurable via a Graphical User Interface (GUI)
- how to build a GUI tool to inspect the trained models in the same Jupyter notebook
- practical example codes to experiment these tools on their own datasets and models

![Geoviews-RGB-Mask](../assets/hv-rgb-and-mask.png)

Figure 1: Using Geoviews to visualize RGB and Mask images. Overlay of RGB and Mask images. Users can hover over the plot to check the RGB values, Lat/Lon and road types

---

![](../assets/gui-osm-downloader.png)

Figure 2: Using `panel`, `param` to create a GUI to link external OSM data   
        Left: Satellite RGB and Segmentation Mask. Right: OSM roadlines


## Outline

This tutorial will consists of five main sections. I will first introduce the basic concepts behind `Holoviews/Geoviews` and `Panel` which are the main libraries we are going to use to add interactive exploration tools for data exploration and model training/evaluation, all in a single Jupyter notebook.  This will take ~15 minutes.  The rest of the tutorial will flow in the order of the general neural network training workflow, while integrating these libraries at each step. I will leave the last <10 minutes for questions.

- Step 0: Introduction to `Holoviews`/`Geoviews` and `Panel` [15mins]
- Step 1: Explore your dataset with `Holoviews`/`Geoviews` [15mins]
- Step 2: Build an easily-configurable neural network model with `param` [15mins]
- Step 3: Monitor your training process with an interactive GUI [15mins]
- Step 4: Analyze your learned model on new images + Understand what your model has learned by looking at intermediate feature maps with `Holoviews` and `Panel` [15mins]
- Q/A [5~10 mins]




<---#todo: remove?
The general workflow of training a neural network goes as following:
1. Prepare your dataset
2. Choose your architecture and training configurations 
    - eg: learning rate, weight decay, number of epoches, optimizer configs
2. Train your model on train data
    - Monitor training by keeping track of the model's performances on validation data
3. Analyze your trained model on test data
    - Understand what your model has learned in a deeper way by,
    - eg. inspecting the intermediate layers (ie. "feature maps")

--->
Throughout this talk, I will show how we can use pyViz tools to add interactive explorations into this workflow without much difficulties. These tools make it more intuitive and easy to explore your datasets, control the training process and inspect what is happening at each step, and understand the test outputs and the learned model in a deeper way. 

## Step 0: Introduction to `PyViz` libraries
In this introductory section, I will go over the basic concepts behind the `PyViz` libraries which are the workhorses of the tutorial. For each of the following libraries in the PyViz suite, I will provide simple examples that show how we can replace any `Matplotlib` plot calls with `Holoviews/Geoviews` with no hassle, and build easy tools to interact with your data.

The core idea behind `Holoviews` is to separate the semantic data from its visualization.
#todo -read from the document and imporve this explanation

## Step 1: Explore your dataset

The first step in building a machine learning model is to understand your dataset. For the scope of this tutorial (ie.semantic segmentation of road types from satellite images), we will use the SpaceNet datasets. More details on how to get the data as well as how the data are collected and annotated can be found [here](https://spacenetchallenge.github.io/datasets/datasetHomePage.html). The original dataset are very large (>100GB) and requires a lot of preprocessing to be useful for training. For example, the RGB images are 16bits of size 1300x1300, and the "target" roads are vector lines (as opposed to raster images), which means they need to be rasterized. I have already prepared a smaller sample dataset consisting of the RGB images converted to 8bits and cropped to 520x520 size, as well as road buffers as rasters which can be easily used as the target images. I will share the dataset to accompany my tutorial.  The shared dataset will consists of input RGB images and target mask images. Each pixel of a target image will contain one of the labels in {'highway', 'track', 'dirt', 'others'} (as `uint8`).

The focus of this section is to show how to build a GUI-like visualization of a satellite dataset within a Jupyter notebook using `Holoviews`/`Geoviews`. See Figure 1 for an example.  Unlike a static plot (eg. generated from Matplotlib), one can hover over the plot to inspect the label values at each pixel of the mask image and check the lat/lon locations. Furthermore I will show how you can trigger more complicated computations (eg. compute road length within a selected zone), while interacting with the plot (eg. selecting a region by mouse drag, clicking a lat/lon by mouse click)

The second example will show how this interactive plot can extended to incorporate external information (eg. roadlines from OpenStreetMap) to easily compare with your own dataset. See Figure 2 for a snapshot of such tool.  In this example, as you select different RGB filenames (of your dataset), you have an option to click on the 'click to download OSM' to download the corresponding region's OSM road data, and visualize it as an interactive map.



## Step 2: Monitor the training process 
In this section, I will show how to wrap around a `PyTorch`'s NN model with `param` so that we can control the (hyper)parameter configurations as well as training processes such as learning rate and batch sizes. 



## Step 3: Interactively test your trained model on the new data

## Step 4: What have the model learned?

## Step 5: Examples

## Conclusion:  Summarize key points
- Main Takeway

- Resources
    - General: 
        - Github repo for this talk:
        - PyViz libraries:
            - Holoviews, Geoviews, Panel, Param
            - scipy talk
            - more: DataShader
            
        - PyTorch:
            - torchvision

    - Data:
        - Remote sensing data: google-earth-engine
        - xarray, dash, 
        - Spatial data: rasterio, geopandas (,xarray)
        

---
## Archive


- Prepare your dataset: train, validation, test
    - classification: 
        - eg: airplane/not-airplane, cat/dog/giraffe, land cover classifiation (forest, road, ...)
        - eg: semantic segmentation: classify each pixel into a label in the label categories
    - clustering: 
    
This talk focuses on the semantic segmentation. So our dataset consists of the input image (RGB) and the target image which will be a "mask" image whose pixel has one of the labels in {'highway', 'track', 'dirt', 'others'}


## Archive

You will see that this allows which in return give your more control over your experiments by exposing model parameters as explicitly tunable parameters of a simple GUI that we are going to build. This GUI can be thought of as the state your model or of your training session.  I will demonstrate how to 






whil, and explore  and as well as directly tune your training hyperparameters (such as learning rate, optimizer configurations, such interacti Holoviews/Geoviews can help explore your dataset and monitor the training process more interactively. To show the benefits, I will show the comparisons with other libraries like Matplotlib.  In addition, I will show how the `param` library facilitates interactive (hyper)parameter tuning when we define a nerual network as a subclass of `param.Parametrized`. Compared to conventional ways to specify the hyperparameter settings (eg. via 'argparse' library or config files), this way of defining a model allows you to change and experiment the hyperparameters within the **same**  Jupyter notebook via a **graphical user interface** (GUI).  To guide through these steps, I will focus on the problem of classifying different road types on satellite images, defined as a multi-class semantic segmentation problem.  Starting from the data exploration to the trained model inspection, the audiences will learn different ways to explore their data and models by easily building simple GUIs in the same Jupyter notebook.

In summary, by the end of the talk the audience will have learned:
- how to make their data exploration more intuitive and experimental by using HoloViz libraries
- how to turn their codes into a simple GUI that allows the above
- how to that exposes 
the benefits of using `Holoviews`/`Geoviews` for interactively exploring datasets and monitoring their training process
- how to use `param` and `panel` to define a neural network whose parameters are configurable via a Graphical User Interface (GUI)
- how to build a GUI tool to inspect the trained models in the same Jupyter notebook
- practical example codes to experiment these tools on their own datasets and models